In [1]:

# 城市名称
# 机场名称
# 航司名称
# 飞机型号
# 飞机注册号
from utils import *
from decimal import *
import pandas as pd
from tqdm import tqdm
from Q_templates import Q_templates_easy
import json


class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Decimal):
            return float(o)
        super(DecimalEncoder, self).default(o)


connection = connect_to_database("ai_database_dev")
cursor = connection.cursor()
connection1 = connect_to_database("data_center_dev")
cursor1 = connection1.cursor()
connection2 = connect_to_database("data_center_release")
cursor2 = connection2.cursor()

In [ ]:
# 2-30
Q_template = "提供{year}年每月各个国家的飞行量及其环比增长或下降百分比。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/country_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
country_dict_list = df[['country_name', 'country_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                a.dep_country_name,
                count(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
                AND a.dep_country_name IS NOT NULL
        GROUP BY
                a.date_month,
                a.dep_country_name
        ),
        biao_2 AS ( SELECT *, lag( num ) over ( PARTITION BY dep_country_name ORDER BY date_month ) AS last_num FROM biao_1 ) SELECT
        date_month,
        dep_country_name,
        num,
        last_num,
        concat( round(( num - last_num )/ last_num * 100, 2 ), '%' ) AS rate
FROM
        biao_2
WHERE
        last_num IS NOT NULL;'''
years = [2024]
for year in years:
    SQL = SQL_template.format(year,year)
    Q = Q_template.replace("{year}", str(year))
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except:
        continue
    print(A)
    if A:
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-30.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-31
Q_template = "列出{year}年各月使用最多的前五个飞机型号，并提供环比变化。"
DB = 'ai_database_dev'
# supplier_csv = "meta_data/aircraft_model_name2id.csv"
# df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
# country_dict_list = df[['country_name', 'country_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                b.aircraft_model_id,
                COUNT(*) AS num
        FROM
                ai_database_dev.zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
        GROUP BY
                a.date_month,
                b.aircraft_model_id
        ),
        biao_2 AS ( SELECT date_month, aircraft_model_id, num, LAG( num ) OVER ( PARTITION BY aircraft_model_id ORDER BY date_month ) AS last_num FROM biao_1 ),
        biao_3 AS ( SELECT date_month, aircraft_model_id, num, last_num, ROW_NUMBER() OVER ( PARTITION BY date_month ORDER BY num DESC ) AS rn FROM biao_2 ) SELECT
        date_month,
        aircraft_model_id,
        num,
        last_num,
        concat( round(( num - last_num )/ last_num * 100, 2 ), '%' ) AS rate
FROM
        biao_3
WHERE
        rn <= 5
        AND last_num IS NOT NULL
ORDER BY
        date_month,
        rn;'''
years = [2024]
for year in years:
    SQL = SQL_template.format(year,year)
    Q = Q_template.replace("{year}", str(year))
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except:
        continue
    print(A)
    if A:
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-31.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [2]:
# 2-32
Q_template = "列出{year}年{month1}月到{month2}月的飞行量增幅最大？提供增幅百分比。"
DB = 'ai_database_dev'
# supplier_csv = "meta_data/aircraft_model_name2id.csv"
# df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
# country_dict_list = df[['country_name', 'country_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                b.aircraft_model_id,
                count(*) AS last_num
        FROM
                ai_database_dev.zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month = '{}-{}'
        GROUP BY
                a.date_month,
                b.aircraft_model_id
        ),
        biao_2 AS (
        SELECT
                a.date_month,
                b.aircraft_model_id,
                count(*) AS num
        FROM
                ai_database_dev.zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month = '{}-{}'
        GROUP BY
                a.date_month,
                b.aircraft_model_id
        ) SELECT
        a.aircraft_model_id,
        a.last_num,
        b.num,(
                b.num - a.last_num
        ) AS diff_num,
        concat( round(( b.num - a.last_num )/ a.last_num * 100, 2 ), '%' ) AS rate
FROM
        biao_1 a
        JOIN biao_2 b ON a.aircraft_model_id = b.aircraft_model_id
ORDER BY
        ( b.num - a.last_num ) DESC
        LIMIT 5;'''
years = [2024]
for year in years:
    for month1 in ['01', '02', '03', '04']:
        for month2 in ['07', '08', '09', '10']:
            SQL = SQL_template.format(year,month1,year,month2)
            Q = Q_template.replace("{year}", str(year))
            Q = Q.replace("{month1}", str(month1))
            Q = Q.replace("{month2}", str(month2))
            print(Q)
            A = None
            try:
                cursor.execute(SQL)
                A = cursor.fetchall()
            except:
                continue
            print(A)
            if A:
                res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
                dataset.append(res_data)
                output_path = 'synthetic_data/2-32.json'
                with open(output_path, "w", encoding="utf-8") as f:
                    json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

列出2024年01月到07月的飞行量增幅最大？提供增幅百分比。
(('449833', 12245, 15086, 2841, '23.20%'), ('719597', 13420, 16078, 2658, '19.81%'), ('444718', 10636, 13066, 2430, '22.85%'), ('568693', 7592, 9283, 1691, '22.27%'), ('995197', 14793, 16290, 1497, '10.12%'))
列出2024年01月到08月的飞行量增幅最大？提供增幅百分比。
(('449833', 12245, 15412, 3167, '25.86%'), ('719597', 13420, 15783, 2363, '17.61%'), ('444718', 10636, 12510, 1874, '17.62%'), ('995197', 14793, 16002, 1209, '8.17%'), ('113033', 3181, 4288, 1107, '34.80%'))
列出2024年01月到09月的飞行量增幅最大？提供增幅百分比。
(('719597', 13420, 15998, 2578, '19.21%'), ('995197', 14793, 17064, 2271, '15.35%'), ('449833', 12245, 14235, 1990, '16.25%'), ('444718', 10636, 12434, 1798, '16.90%'), ('332441', 7017, 8093, 1076, '15.33%'))
列出2024年01月到10月的飞行量增幅最大？提供增幅百分比。
(('821692', 16, 25, 9, '56.25%'), ('854724', 4, 12, 8, '200.00%'), ('398544', 47, 53, 6, '12.77%'), ('674065', 19, 24, 5, '26.32%'), ('479833', 145, 149, 4, '2.76%'))
列出2024年02月到07月的飞行量增幅最大？提供增幅百分比。
(('449833', 12496, 15086, 2590, '20.73%'), ('56

In [ ]:
# 2-33
Q_template = "提供{year}年每月{supplier_name}的各机型平均飞行小时数。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''SELECT
        a.date_month,
        b.aircraft_model_id,
        b.aircraft_model_name,
        round( AVG( a.diff_minutes ) / 60, 2 ) AS avg_flight_hours
FROM
        zts_plane_flight_1 a
        JOIN aircraft_details b ON a.pure_reg = b.pure_reg
WHERE
        b.supplier_id = "{}"
        AND a.date_month BETWEEN "{}-01"
        AND "{}-12"
GROUP BY
        a.date_month,
        b.aircraft_model_id,
        b.aircraft_model_name
ORDER BY
        a.date_month,
        b.aircraft_model_id;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:50]):
    for year in years:
        for i in range(3):
            SQL = SQL_template.format(supplier['supplier_id'], year,year)
            Q = Q_template.replace("{year}", str(year))
            Q = Q.replace("{supplier_name}", supplier['supplier_name'])
            # print(Q)
            A = None
            try:
                cursor.execute(SQL)
                A = cursor.fetchall()
            except:
                continue
            print(A)
            if A:
                res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
                dataset.append(res_data)
                output_path = 'synthetic_data/2-33.json'
                with open(output_path, "w", encoding="utf-8") as f:
                    json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-34
Q_template = "列出{year}年飞行量最多的机型及每月变化情况。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                b.type_code,
                COUNT(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN aircraft_details b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
        GROUP BY
                a.date_month,
                b.type_code
        ),
        most_frequent_type AS (
        SELECT
                b.type_code
        FROM
                zts_plane_flight_1 a
                JOIN aircraft_details b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
        GROUP BY
                b.type_code
        ORDER BY
                COUNT(*) DESC
                LIMIT 1
        ),
        biao_2 AS ( SELECT b1.date_month, b1.num FROM biao_1 b1 JOIN most_frequent_type mft ON b1.type_code = mft.type_code ),
        biao_3 AS ( SELECT *, lag( num ) over ( ORDER BY date_month ) AS last_num FROM biao_2 ) SELECT
        *,
        num - last_num AS diff_num,
        concat( round(( num - last_num )/ last_num * 100, 2 ), '%' ) AS rate
FROM
        biao_3
WHERE
        last_num IS NOT NULL
ORDER BY
        date_month;'''
years = [2024]
for year in years:
    SQL = SQL_template.format(year,year,year,year)
    Q = Q_template.replace("{year}", str(year))
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except:
        continue
    print(A)
    if A:
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-34.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [4]:
# 2-35
Q_template = "{aircraft_model_name}机型在{year}年每个月的飞行量变化如何？"
DB = 'ai_database_dev'
aircraft_csv = "meta_data/aircraft_model_name2id.csv"
df = pd.read_csv(aircraft_csv)
dataset = []
# 获取所有的supplier_id列的值
aircraft_dict_list = df[['aircraft_model_name', 'aircraft_model_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                count(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
                AND b.aircraft_model_id = "{}"
        GROUP BY
                a.date_month
        ),
        biao_2 AS ( SELECT *, lag( num ) over ( ORDER BY date_month ) AS last_num FROM biao_1 ) SELECT
        *,
        num - last_num AS diff_num
FROM
        biao_2
ORDER BY
        date_month;'''
years = [2024]
for aircraft in aircraft_dict_list[:50]:
    for year in years:
        SQL = SQL_template.format(year,year,aircraft['aircraft_model_name'])
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{aircraft_model_name}", aircraft['aircraft_model_name'])
        print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-35.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)
        # if A:
        #     res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        #     dataset.append(res_data)
        #     output_path = 'synthetic_data/2-35.json'
        #     with open(output_path, "w", encoding="utf-8") as f:
        #         json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

Embraer Phenom 300机型在2024年每个月的飞行量变化如何？
()
Embraer Phenom 300机型在2024年每个月的飞行量变化如何？
()
Bombardier Global 6000机型在2024年每个月的飞行量变化如何？
()
Bombardier Challenger 850机型在2024年每个月的飞行量变化如何？
()
Bombardier Challenger 850机型在2024年每个月的飞行量变化如何？
()
Bombardier Global 6000机型在2024年每个月的飞行量变化如何？
()
Gulfstream G450机型在2024年每个月的飞行量变化如何？
()
Bombardier Challenger 605机型在2024年每个月的飞行量变化如何？
()
Bombardier Challenger 605机型在2024年每个月的飞行量变化如何？
()
Bombardier Global Express机型在2024年每个月的飞行量变化如何？
()
Bombardier Global 6000机型在2024年每个月的飞行量变化如何？
()
Bombardier Challenger 605机型在2024年每个月的飞行量变化如何？
()
Bombardier Global 6000机型在2024年每个月的飞行量变化如何？
()
Bombardier Global 6000机型在2024年每个月的飞行量变化如何？
()
Bombardier Global 6000机型在2024年每个月的飞行量变化如何？
()
Gulfstream G450机型在2024年每个月的飞行量变化如何？
()
Gulfstream G450机型在2024年每个月的飞行量变化如何？
()
Dassault Falcon 7X机型在2024年每个月的飞行量变化如何？
()
Gulfstream G450机型在2024年每个月的飞行量变化如何？
()
Bombardier Challenger 605机型在2024年每个月的飞行量变化如何？
()
Dassault Falcon 8X机型在2024年每个月的飞行量变化如何？
()
Bombardier Challenger 605机型在2024年每个月的飞行量变化如何？
()
Bombardi

In [ ]:
# 2-40
Q_template = "{airport_name}机场在2024年各月的飞行量（起降一起），并提供2024年每个月环比上一个月的增长率。（2024年1月环比2023年12月）"
DB = 'ai_database_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month COLLATE utf8mb4_general_ci AS date_month,
                COUNT(*) AS dep_count
        FROM
                zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.dep_icao = "{}" COLLATE utf8mb4_general_ci
                AND a.date_month BETWEEN '2023-12' COLLATE utf8mb4_general_ci
                AND '2024-12' COLLATE utf8mb4_general_ci
        GROUP BY
                a.date_month COLLATE utf8mb4_general_ci
        ),
        biao_2 AS (
        SELECT
                a.date_month COLLATE utf8mb4_general_ci AS date_month,
                COUNT(*) AS arr_count
        FROM
                zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.arr_icao = "{}" COLLATE utf8mb4_general_ci
                AND a.date_month BETWEEN '2023-12' COLLATE utf8mb4_general_ci
                AND '2024-12' COLLATE utf8mb4_general_ci
        GROUP BY
                a.date_month COLLATE utf8mb4_general_ci
        ),
        biao_3 AS (
        SELECT
                a.date_month,
                ( a.dep_count + b.arr_count ) AS num
        FROM
                biao_1 a
                JOIN biao_2 b ON a.date_month = b.date_month
        ),
        biao_4 AS ( SELECT date_month, num, lag( num ) over ( ORDER BY date_month ) AS last_num FROM biao_3 ) SELECT
        date_month,
        num,
        last_num,
        concat( round(( num - last_num )/ last_num * 100, 2 ), '%' ) AS rate
FROM
        biao_4
WHERE
        last_num IS NOT NULL;'''
years = [2024]
for airport in airport_dict_list[:100]:
    for year in years:
        SQL = SQL_template.format(airport['icao_code'],airport['icao_code'])
        Q = Q_template.replace("{airport_name}", airport['airport_name'])
        print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        if A:
            res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
            dataset.append(res_data)
            output_path = 'synthetic_data/2-40.json'
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)